# Choosing between multiple schemas
The simplest way to let the model choose from multiple schemas is to create a parent schema that has a Union-typed attribute:



In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Response(output=Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=7))

In [7]:
from typing import Union
from langchain_core.pydantic_v1 import BaseModel, Field
from typing_extensions import Annotated, TypedDict, Optional


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")



In [8]:

class Response(BaseModel):
    output: Union[Joke, ConversationalResponse]


In [9]:


structured_llm = llm.with_structured_output(Response)


In [10]:

structured_llm.invoke("Tell me a joke about cats")

Response(output=Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=7))

In [11]:
structured_llm.invoke("How are you today?")

Response(output=ConversationalResponse(response="I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How about you? How's your day going?"))

# Streaming

In [12]:
from typing_extensions import Annotated, TypedDict


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

for chunk in structured_llm.stream("Tell me a joke about cats"):
    print(chunk)

{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why did'}
{'setup': 'Why did the'}
{'setup': 'Why did the cat'}
{'setup': 'Why did the cat sit'}
{'setup': 'Why did the cat sit on'}
{'setup': 'Why did the cat sit on the'}
{'setup': 'Why did the cat sit on the computer'}
{'setup': 'Why did the cat sit on the computer?'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': ''}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'Because'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'Because it'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'Because it wanted'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'Because it wanted to'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'Because it wanted to keep'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'Because it wanted to keep an'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'Because it wanted to keep an eye'}
{'setup'